In [1]:
import pandas as pd 
import numpy as np
from scipy import stats

import helpers

## EDA and Data Prep

In [2]:
prequals = pd.read_csv('prequals.csv', index_col=0)
intellicron_prequals = pd.read_csv('intellicron_prequals.csv', index_col=0)
print(f'prequals length: \t\t{len(prequals)}\nintellicron_prequals length: \t{len(intellicron_prequals)}')

prequals length: 		190830
intellicron_prequals length: 	8495


In [3]:
print('prequals:')
display(prequals.head(3))
print('intellicron_prequals:')
display(intellicron_prequals.head(3))

prequals:


,prequal_id,checkout_id,prequal_date,completed_prequal,approved
1,0683430d-2d5d-4206-84b8-c7226540e9c9,NaN,2019-01-31,0,0
2,9af59e9b-bedb-4718-bfac-27c58529bc09,NaN,2019-02-09,1,0
3,46b5c524-a1e7-4e52-9d05-d9cdffde6634,NaN,2019-06-21,1,0


intellicron_prequals:


,prequal_id,assignment_date
1,0ed3973c-830c-4dc8-9789-78cad38fefef,2019-06-17
2,cdadc46a-0937-43a6-9a57-16502c04b772,2019-06-18
3,ad29bbb9-6698-45c8-97bb-f389e99e396e,2019-06-16


In [4]:
print('prequals: \t\t', helpers.get_date_range(prequals, 'prequal_date'))
print('intellicron_prequals: \t', helpers.get_date_range(intellicron_prequals, 'assignment_date'))

prequals: 		 2019-01-01 to 2019-06-30
intellicron_prequals: 	 2019-06-16 to 2019-06-30


- Above we notice that the begining date of the prequals data starts 6 and a hal months earlier than the start of the test
- This statement is assuming that 'prequal_date' and 'assignment_date' are the same

In [5]:
variant_results = pd.merge(prequals, intellicron_prequals, on='prequal_id')

date_diffs = len(variant_results[variant_results.assignment_date!=variant_results.prequal_date])
print(f'Differences between "prequal_date" and "assignment_date": {date_diffs}')

variant_results = variant_results.drop('assignment_date', axis=1)

Differences between "prequal_date" and "assignment_date": 0


- Based on the above we can assume that 'prequal_date' and 'assignment_date' are the same
- To avoid using data that was not in the test range, we'll filter the non intellicron results to be of the same date range

In [6]:
#filter out prequal ids
control_results_total = prequals[~prequals.prequal_id.isin(variant_results.prequal_id)]
#filter out earlier dates
control_results = control_results_total[control_results_total.prequal_date>=variant_results.prequal_date.min()]
print('control_results: \t', helpers.get_date_range(control_results, 'prequal_date'))
print('variant_results: \t', helpers.get_date_range(variant_results, 'prequal_date'))

control_results: 	 2019-06-16 to 2019-06-30
variant_results: 	 2019-06-16 to 2019-06-30


In [7]:
print(f'control_results length: \t{len(control_results)}\nvariant_results length: \t{len(variant_results)}')

control_results length: 	9690
variant_results length: 	8495


- After date filtering we see a pool that makes more sense being that our users were split 50/50
- Now that we have split the data into two result sets we can combine into one datasetremove unecessary variables and start analysis

In [8]:
#add labels then combine results and drop uneeded variables
control_results.loc[:,'group'] = 'A'
variant_results.loc[:,'group'] = 'B'

ab = pd.concat([control_results, variant_results]).drop(['prequal_date', 'completed_prequal', 'approved', 'prequal_id'], axis=1)

#setup checkout column
ab.loc[ab.checkout_id.isnull(),'checkout_id'] = 0
ab.loc[~(ab.checkout_id==0),'checkout_id'] = 1
ab.columns = ['converted', 'group']

/Users/devonnavon/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/devonnavon/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


## Analysis
- Below we will look at difference in conversion rate 

In [9]:
ab_summary = ab.pivot_table(values='converted', index='group', aggfunc=np.sum)
ab_summary['total'] = ab.pivot_table(values='converted', index='group', aggfunc=lambda x: len(x))
ab_summary['rate'] = ab_summary['converted']/ab_summary['total']

In [10]:
diff = ab_summary.loc['B','rate'] - ab_summary.loc['A','rate']
print(f'\nGroup B had a {diff.round(4)} higher conversion rate\n')
ab_summary


Group B had a 0.0091 higher conversion rate



,converted,total,rate
group,,,
A,2543,9690,0.262436
B,2307,8495,0.271572


While group B has a higher conversion rate we need to test the significance of this with the following hypothesis

    Ho : Conversion Rate of Group A ≥ Conversion Rate of Group B   
    Ha : Conversion Rate of Group A < Conversion Rate of Group B

We will run this expiriemnt at the 5% significance level to determine if we can reject the null hypothesis   
Because we are comparing two samples we run a t-test and divide the p-value by 2 in order to get the one sided probability

In [11]:
pop_1 = ab.loc[ab.group=='A', 'converted']
pop_2 = ab.loc[ab.group=='B', 'converted']

results = stats.ttest_ind(pop_1, pop_2)
print(f'p-value: {(results.pvalue/2).round(4)}')

p-value: 0.0823


The p-value of .0823 is less than .05 so we cannot say with 95% confidence that the results are significant 